In [1]:
import sys
sys.version

'3.10.6 (tags/v3.10.6:9c7b4bd, Aug  1 2022, 21:53:49) [MSC v.1932 64 bit (AMD64)]'

In [2]:
! pip install -q pandas
! pip install -q  nltk


[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import os
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from datetime import datetime
from typing import Union



[nltk_data] Downloading package punkt to C:\Users\Candace
[nltk_data]     Edwards\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
#
#  Import raw data
#

#source =r'..\data\aacsb\raw\standards\Data - STANDARDS.csv'
source =r'..\data\aacsb\raw\standards\Data - SECTIONS.csv'
df = pd.read_csv(source)
df

,Section_Num,Section_Title,Section_Description
0,1,Strategic Managemend and Innovation,Strategic management encompasses the entire ra...
1,2,Learner Success,This section of the accreditation standards is...
2,3,"Though Leadership, Engagement, and Societal Im...",The defining feature of quality business schoo...


In [11]:
df.fillna(value=str(None), inplace=True) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Section_Num          3 non-null      int64 
 1   Section_Title        3 non-null      object
 2   Section_Description  3 non-null      object
dtypes: int64(1), object(2)
memory usage: 200.0+ bytes


In [12]:
def preprocess_text(text: str) -> str:
    """
    Preprocesses text by converting it to lowercase, removing special characters
    except periods and new lines, tokenizing the text, removing stopwords, and 
    joining the tokens back into a single string.

    Parameters:
    text (str): The input text to be preprocessed.

    Returns:
    str: The preprocessed text.
    """

    text = text.lower()

    #remove special chars except: period and newlines -- needed for downstream chunking
    text = re.sub(r'[^a-zA-Z0-9.\n\s]', ' ', text)

    tokens = word_tokenize(text)

    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    processed_text = ' '.join(filtered_tokens)

    return processed_text

In [13]:
test_text = "We are going to Kentucky and going to the fair to see the senorita with flowers • in •  her hair . Time to go"
print(preprocess_text(test_text))

going kentucky going fair see senorita flowers hair . time go


In [14]:
#
# Pre-process dataframe text
#


processed_df = df.copy()
#columns_to_process = ['Standard_formal', 'Definitions', 'Basis_for_judgement','Supporting_docs']
columns_to_process = ['Section_Description']

processed_df[columns_to_process] = processed_df[columns_to_process].map(preprocess_text)
processed_df.head()

,Section_Num,Section_Title,Section_Description
0,1,Strategic Managemend and Innovation,strategic management encompasses entire range ...
1,2,Learner Success,section accreditation standards designed ensur...
2,3,"Though Leadership, Engagement, and Societal Im...",defining feature quality business schools maki...


In [15]:
def save_dataframe_to_json(df: pd.DataFrame, filename: str) -> None:
    """
    Save DataFrame to JSON file with current datetime appended to the filename.

    Parameters:
    df (pd.DataFrame): The DataFrame to be saved.
    filename (str): The output filename without extension.
    """
    # Specify the path
    path = r'..\data\aacsb\processed'
    
    # Generate filename with datetime
    current_datetime = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    output_filename = f"{filename}_processed_{current_datetime}.json"
    
    # Create output directory if it doesn't exist
    os.makedirs(path, exist_ok=True)
    
    # Save DataFrame to JSON file
    output_path = os.path.join(path, output_filename)
    df.to_json(output_path, orient='records')
    
    print(f"DataFrame saved to {output_path}")

In [16]:
#
# Convert to JSON and Export
#

file_name = 'SECTIONS'
save_dataframe_to_json(processed_df, file_name)

DataFrame saved to ..\data\aacsb\processed\SECTIONS_processed_2024-04-04_16-24-04.json
